In [1]:
import keras.layers as KL
from keras.models import Model
import keras.backend as K
import tensorflow as tf
import numpy as np

Using TensorFlow backend.


In [2]:
def building_block(filters, block):
    if block !=0 :
        stride = 1
    else:
        stride = 2
    def f(x):
        y = KL.Conv2D(filters, (1,1), strides=stride)(x)
        y = KL.BatchNormalization(axis=3)(y)
        y = KL.Activation("relu")(y)
        
        y = KL.Conv2D(filters, (3,3), padding="same")(y)
        y = KL.BatchNormalization(axis=3)(y)
        y = KL.Activation("relu")(y)
        
        y = KL.Conv2D(4*filters, (1,1))(y)
        y = KL.BatchNormalization(axis=3)(y)
        
        if block == 0:
            shorcut = KL.Conv2D(4*filters, (1,1), strides=stride)(x)
            shorcut = KL.BatchNormalization(axis=3)(shorcut)
        else:
            shorcut = x
        y = KL.Add()([y, shorcut])
        y = KL.Activation("relu")(y)
        return y
    return f

In [3]:
def resNet_featureExtractor(inputs):
    x = KL.Conv2D(64, (3,3), padding="same")(inputs)
    x = KL.BatchNormalization(axis=3)(x)
    x = KL.Activation("relu")(x)
    
    filters = 64
    blocks = [3, 6, 4]
    for i, block_num in enumerate(blocks):
        for block_id in range(block_num):
            x = building_block(filters, block_id)(x)
        filters *= 2
    return x

In [4]:
def rpn_net(inputs, k):
    shared_map = KL.Conv2D(256, (3,3), padding="same")(inputs)
    shared_map = KL.Activation("linear")(shared_map)
    rpn_class = KL.Conv2D(2*k, (1,1))(shared_map)
    rpn_class = KL.Lambda(lambda x: tf.reshape(x, [tf.shape(x)[0],-1,2]))(rpn_class)
    rpn_class = KL.Activation("linear")(rpn_class)
    rpn_prob = KL.Activation("softmax")(rpn_class)
    
    y = KL.Conv2D(4*k, (1,1))(shared_map)
    y = KL.Activation("linear")(y)
    rpn_bbox = KL.Lambda(lambda x: tf.reshape(x, [tf.shape(x)[0],-1,4]))(y)
    return rpn_class, rpn_prob, rpn_bbox

In [5]:
x = KL.Input((64, 64, 3))
fp = resNet_featureExtractor(x)
rpn_class, rpn_prob, rpn_bbox = rpn_net(fp, 9)
model = Model([x],[rpn_class, rpn_prob, rpn_bbox])   ## forward
#model.summary()

In [5]:
def rpn_class_loss(rpn_match, rpn_class_logits):
    ## rpn_match (None, 576, 1)
    ## rpn_class_logits (None, 576, 2)
    rpn_match = tf.squeeze(rpn_match, -1)
    indices = tf.where(K.not_equal(rpn_match, 0))
    anchor_class = K.cast(K.equal(rpn_match, 1), tf.int32)
    rpn_class_logits = tf.gather_nd(rpn_class_logits, indices)     ### prediction
    anchor_class = tf.gather_nd(anchor_class, indices)   ### target
    loss = K.sparse_categorical_crossentropy(target=anchor_class, output=rpn_class_logits, from_logits=True)
    loss = K.switch(tf.size(loss) > 0 , K.mean(loss), tf.constant(0.0))
    return loss

def batch_back(x, counts, num_rows):
    outputs = []
    for i in range(num_rows):
        outputs.append(x[i, :counts[i]])
    return tf.concat(outputs, axis=0)


def rpn_bbox_loss(target_bbox, rpn_match, rpn_bbox):
    rpn_match = tf.squeeze(rpn_match, -1)
    indices = tf.where(K.equal(rpn_match, 1))
    rpn_bbox = tf.gather_nd(rpn_bbox, indices)
    batch_counts = K.sum(K.cast(K.equal(rpn_match, 1), tf.int32), axis=1)
    target_bbox = batch_back(target_bbox, batch_counts, 20)
    diff = K.abs(target_bbox - rpn_bbox)
    less_than_one = K.cast(K.less(diff, 1.0), "float32")
    loss = (less_than_one * 0.5 * diff**2) + (1 - less_than_one) * (diff - 0.5)
    loss = K.switch(tf.size(loss) > 0 , K.mean(loss), tf.constant(0.0))
    return loss

In [6]:
input_image = KL.Input(shape=[64,64,3], dtype=tf.float32)
input_bboxes = KL.Input(shape=[None,4], dtype=tf.float32)
input_class_ids = KL.Input(shape=[None],dtype=tf.int32)
input_rpn_match = KL.Input(shape=[None, 1], dtype=tf.int32)
input_rpn_bbox = KL.Input(shape=[None, 4], dtype=tf.float32)

In [7]:
feature_map = resNet_featureExtractor(input_image)
rpn_class, rpn_prob, rpn_bbox = rpn_net(feature_map, 9)

loss_rpn_match = KL.Lambda(lambda x: rpn_class_loss(*x), name="loss_rpn_match")([input_rpn_match, rpn_class])

loss_rpn_bbox = KL.Lambda(lambda x: rpn_bbox_loss(*x), name="loss_rpn_bbox")([input_rpn_bbox, input_rpn_match, rpn_bbox])

model = Model([input_image, input_bboxes, input_class_ids, input_rpn_match, input_rpn_bbox],
              [rpn_class, rpn_prob, rpn_bbox, loss_rpn_match, loss_rpn_bbox])



In [8]:
import keras
loss_lay1 = model.get_layer("loss_rpn_match").output
loss_lay2 = model.get_layer("loss_rpn_bbox").output

model.add_loss(tf.reduce_mean(loss_lay1))
model.add_loss(tf.reduce_mean(loss_lay2))

model.compile(loss=[None]*len(model.output), optimizer=keras.optimizers.SGD(lr=0.00005, momentum=0.9))

model.metrics_names.append("loss_rpn_match")
model.metrics_tensors.append(tf.reduce_mean(loss_lay1, keep_dims=True))

model.metrics_names.append("loss_rpn_bbox")
model.metrics_tensors.append(tf.reduce_mean(loss_lay2, keep_dims=True))

In [9]:
from utils import shapeData as dataSet
from config import Config

config = Config()
dataset = dataSet([64,64], config=config)

In [10]:
def data_Gen(dataset, num_batch, batch_size, config):
    for _ in range(num_batch):
        images = []
        bboxes = []
        class_ids = []
        rpn_matchs = []
        rpn_bboxes = []
        for i in range(batch_size):
            image, bbox, class_id, rpn_match, rpn_bbox, _ = data = dataset.load_data()
            pad_num = config.max_gt_obj - bbox.shape[0]
            pad_box = np.zeros((pad_num, 4))
            pad_ids = np.zeros((pad_num, 1))
            bbox = np.concatenate([bbox, pad_box], axis=0)
            class_id = np.concatenate([class_id, pad_ids], axis=0)
        
            images.append(image)
            bboxes.append(bbox)
            class_ids.append(class_id)
            rpn_matchs.append(rpn_match)
            rpn_bboxes.append(rpn_bbox)
        images = np.concatenate(images, 0).reshape(batch_size, config.image_size[0],config.image_size[1] , 3)
        bboxes = np.concatenate(bboxes, 0).reshape(batch_size, -1 , 4)
        class_ids = np.concatenate(class_ids, 0).reshape(batch_size, -1 )
        rpn_matchs = np.concatenate(rpn_matchs, 0).reshape(batch_size, -1 , 1)
        rpn_bboxes = np.concatenate(rpn_bboxes, 0).reshape(batch_size, -1 , 4)
        yield [images, bboxes, class_ids, rpn_matchs, rpn_bboxes],[]

In [11]:
dataGen = data_Gen(dataset, 35000, 20, config)

In [12]:
his = model.fit_generator(dataGen, steps_per_epoch=20, epochs=1600)

Epoch 1/2
20/20 [==============================] - 12s 585ms/step - loss: 2.8941 - loss_rpn_match: 0.8701 - loss_rpn_bbox: 2.0240
Epoch 2/2
20/20 [==============================] - 3s 134ms/step - loss: 2.1590 - loss_rpn_match: 0.4205 - loss_rpn_bbox: 1.7385
